# Loop 5 Analysis: Strategy Assessment

## Key Questions:
1. What is the gap between our best score and the target?
2. What approaches have NOT been tried?
3. What can we learn from the overlapping snapshot?
4. Can we run the C++ optimizer on our current best?

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os
import glob

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])
BASE_TREE = Polygon(zip(TX, TY))

def create_tree(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

def get_side(trees):
    if not trees:
        return 0
    min_x = min_y = float('inf')
    max_x = max_y = float('-inf')
    for tree in trees:
        bounds = tree.bounds
        min_x = min(min_x, bounds[0])
        min_y = min(min_y, bounds[1])
        max_x = max(max_x, bounds[2])
        max_y = max(max_y, bounds[3])
    return max(max_x - min_x, max_y - min_y)

def check_overlap(trees, tolerance=1e-9):
    for i in range(len(trees)):
        for j in range(i + 1, len(trees)):
            if trees[i].intersects(trees[j]):
                intersection = trees[i].intersection(trees[j])
                if intersection.area > tolerance:
                    return True
    return False

print('Utilities loaded')

In [ ]:
# Load current best submission
df_current = pd.read_csv('/home/submission/submission.csv')

# Calculate per-N scores
per_n_scores = {}
for n in range(1, 201):
    prefix = f'{n:03d}_'
    n_rows = df_current[df_current['id'].str.startswith(prefix)]
    trees = []
    for _, row in n_rows.iterrows():
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        trees.append(create_tree(x, y, deg))
    side = get_side(trees)
    per_n_scores[n] = side**2 / n

total_score = sum(per_n_scores.values())
print(f'Current total score: {total_score:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)')

In [ ]:
# Analyze per-N efficiency
print('\nPer-N Efficiency Analysis (worst 20):')
print('=' * 60)

# Theoretical minimum per tree = tree_area = 0.245625
tree_area = BASE_TREE.area
print(f'Tree area: {tree_area:.6f}')

efficiencies = []
for n, score in per_n_scores.items():
    # Efficiency = theoretical_min / actual
    theoretical_min = tree_area  # per tree
    efficiency = theoretical_min / score * 100
    efficiencies.append((n, score, efficiency))

# Sort by efficiency (worst first)
efficiencies.sort(key=lambda x: x[2])

print(f'\n{"N":>4} {"Score":>12} {"Efficiency":>10} {"Contribution":>12}')
for n, score, eff in efficiencies[:20]:
    print(f'{n:4d} {score:12.6f} {eff:9.2f}% {score:12.6f}')

In [ ]:
# Load the overlapping snapshot and compare
overlap_snapshot = '/home/nonroot/snapshots/santa-2025/21145966992/submission/submission.csv'
if os.path.exists(overlap_snapshot):
    df_overlap = pd.read_csv(overlap_snapshot)
    
    print('Comparing overlapping snapshot with current best:')
    print('=' * 70)
    
    improvements = []
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        
        # Current best
        n_rows_current = df_current[df_current['id'].str.startswith(prefix)]
        trees_current = []
        for _, row in n_rows_current.iterrows():
            x = float(str(row['x']).replace('s', ''))
            y = float(str(row['y']).replace('s', ''))
            deg = float(str(row['deg']).replace('s', ''))
            trees_current.append(create_tree(x, y, deg))
        score_current = get_side(trees_current)**2 / n
        
        # Overlapping snapshot
        n_rows_overlap = df_overlap[df_overlap['id'].str.startswith(prefix)]
        trees_overlap = []
        for _, row in n_rows_overlap.iterrows():
            x = float(str(row['x']).replace('s', ''))
            y = float(str(row['y']).replace('s', ''))
            deg = float(str(row['deg']).replace('s', ''))
            trees_overlap.append(create_tree(x, y, deg))
        score_overlap = get_side(trees_overlap)**2 / n
        has_overlap = check_overlap(trees_overlap)
        
        if score_overlap < score_current:
            improvement = score_current - score_overlap
            improvements.append((n, score_current, score_overlap, improvement, has_overlap))
    
    print(f'\nFound {len(improvements)} N values where overlapping snapshot is better:')
    print(f'{"N":>4} {"Current":>12} {"Overlap":>12} {"Improve":>10} {"Has Overlap":>12}')
    
    total_potential = 0
    clean_improvements = 0
    for n, curr, ovl, imp, has_ovl in sorted(improvements, key=lambda x: -x[3])[:30]:
        status = 'OVERLAP' if has_ovl else 'CLEAN'
        print(f'{n:4d} {curr:12.6f} {ovl:12.6f} {imp:10.6f} {status:>12}')
        total_potential += imp
        if not has_ovl:
            clean_improvements += imp
    
    print(f'\nTotal potential improvement: {total_potential:.6f}')
    print(f'Clean (no overlap) improvement: {clean_improvements:.6f}')
else:
    print('Overlapping snapshot not found')

In [ ]:
# Check all snapshots for clean improvements
print('\nSearching ALL snapshots for clean improvements...')
print('=' * 70)

snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
snapshots = [d for d in os.listdir(snapshot_dir) if os.path.isdir(os.path.join(snapshot_dir, d)) and d != '.git']

best_per_n = {n: {'score': per_n_scores[n], 'source': 'current', 'has_overlap': False} for n in range(1, 201)}

for snap in snapshots:
    csv_path = os.path.join(snapshot_dir, snap, 'submission', 'submission.csv')
    if not os.path.exists(csv_path):
        continue
    
    try:
        df_snap = pd.read_csv(csv_path)
        
        for n in range(1, 201):
            prefix = f'{n:03d}_'
            n_rows = df_snap[df_snap['id'].str.startswith(prefix)]
            if len(n_rows) != n:
                continue
            
            trees = []
            for _, row in n_rows.iterrows():
                x = float(str(row['x']).replace('s', ''))
                y = float(str(row['y']).replace('s', ''))
                deg = float(str(row['deg']).replace('s', ''))
                trees.append(create_tree(x, y, deg))
            
            has_overlap = check_overlap(trees)
            if has_overlap:
                continue  # Skip overlapping configurations
            
            score = get_side(trees)**2 / n
            
            if score < best_per_n[n]['score'] - 1e-9:
                best_per_n[n] = {'score': score, 'source': snap, 'has_overlap': False}
    except Exception as e:
        continue

# Count improvements
improvements = [(n, per_n_scores[n], best_per_n[n]['score'], best_per_n[n]['source']) 
                for n in range(1, 201) 
                if best_per_n[n]['source'] != 'current']

print(f'Found {len(improvements)} N values with clean improvements from snapshots:')
if improvements:
    total_improvement = sum(per_n_scores[n] - best_per_n[n]['score'] for n, _, _, _ in improvements)
    print(f'Total potential improvement: {total_improvement:.6f}')
    print(f'\n{"N":>4} {"Current":>12} {"Better":>12} {"Improve":>10} {"Source":>15}')
    for n, curr, better, source in sorted(improvements, key=lambda x: -(x[1] - x[2]))[:20]:
        print(f'{n:4d} {curr:12.6f} {better:12.6f} {curr-better:10.6f} {source:>15}')

In [ ]:
# Check if we can run the C++ optimizer
import subprocess

print('Testing C++ optimizer...')
print('=' * 50)

# Check if optimizer exists
if os.path.exists('/home/code/sa_v1_parallel'):
    print('✓ sa_v1_parallel exists')
    
    # Try running it on a small test
    result = subprocess.run(['/home/code/sa_v1_parallel', '--help'], 
                          capture_output=True, text=True, timeout=5)
    print(f'Help output: {result.stdout[:200] if result.stdout else result.stderr[:200]}')
else:
    print('✗ sa_v1_parallel not found')

In [ ]:
# Summary of findings
print('\n' + '=' * 70)
print('SUMMARY OF FINDINGS')
print('=' * 70)

print(f'\n1. Current Score: {total_score:.6f}')
print(f'   Target: 68.919154')
print(f'   Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)')

print(f'\n2. Worst efficiency N values (where most improvement is possible):')
for n, score, eff in efficiencies[:5]:
    print(f'   N={n}: efficiency={eff:.1f}%, score={score:.6f}')

print(f'\n3. Clean improvements from snapshots: {len(improvements)} N values')
if improvements:
    print(f'   Total potential: {total_improvement:.6f}')

print(f'\n4. Key insight: Top kernels use 19+ sources, we have ~7 unique sources')
print(f'   The gap is likely due to missing data sources, not algorithm quality')